# Introduction to Spark

## A Brief History of Big Data

There's been a lot of buzz about **big data** over the last few years, and it's finally become mainstream. Companies like Google and Yahoo! have grown their user bases significantly, and are collecting more information on how people interact with their products. [Moore's law](https://en.wikipedia.org/wiki/Moore%27s_law) and the rapidly [falling cost of storage](https://www.aei.org/wp-content/uploads/2013/04/storage3_f.jpg) have contributed greatly to the big data phenomena.<br>

While software companies got better at collecting massive amounts of data, their ability to analyze and make sense of it didn't keep pace. **Because existing technologies couldn't analyze such large quantities of data, companies like Google, Facebook, Yahoo!, and LinkedIn had to build new paradigms and tools that could do the job**.<br>

Engineers initially tried using bigger and more powerful computers to process the data, but still ran into limits for many computational problems. Along the way, **they developed paradigms like [MapReduce](https://en.wikipedia.org/wiki/MapReduce) that efficiently distribute calculations over hundreds or thousands of computers to calculate the result in parallel**. 

### Hadoop is an open source project that quickly became the dominant processing toolkit for big data.<br>

### Hadoop

**Hadoop consists of a file system (Hadoop Distributed File System, or HDFS) and its own implementation of the MapReduce paradigm**. 
* MapReduce converts computations into Map and Reduce steps that Hadoop can easily distribute over many machines. We'll cover how MapReduce works in greater depth later in this lesson.

**Hadoop made it possible to analyze large data sets, but relied heavily on disk storage (rather than memory) for computation**. While it's inexpensive to store large volumes of data this way, it [makes accessing and processing it much slower](http://www.cnet.com/news/understanding-ram-versus-hard-drive-space-via-an-analogy/).<br>

Hadoop wasn't a great solution for calculations requiring multiple passes over the same data or many intermediate steps, due to the need to write to and read from the disk between each step. This drawback also made Hadoop difficult to use for interactive data analysis, the main task data scientists need to do.<br>

Hadoop also suffered from suboptimal support for the additional libraries many data scientists needed, such as SQL and machine learning implementations. **Once the cost of RAM (computer memory) started to drop significantly, augmenting or replacing Hadoop by storing data in-memory quickly emerged as an appealing alternative.**

## The Spark Revolution

The [UC Berkeley AMP Lab](https://amplab.cs.berkeley.edu/projects/spark-lightning-fast-cluster-computing/) spearheaded groundbreaking work to develop Spark, which uses distributed, in-memory data structures to improve speeds for many data processing workloads by several orders of magnitude. If you're interested in learning more, you can read about [why Spark is a crossover hit for data scientists](http://blog.cloudera.com/blog/2014/03/why-apache-spark-is-a-crossover-hit-for-data-scientists/), or check out some of the original papers on the [Apache Spark homepage](http://spark.apache.org/research.html).

## Resilent Distributed Data Sets (RDDs)

The core data structure in Spark is a resilient distributed data set (RDD). As the name suggests, an **RDD is Spark's representation of a data set that's distributed across the RAM, or memory, of a cluster of many machines**. 
* An RDD object is essentially a collection of elements we can use to hold lists of tuples, dictionaries, lists, etc. 
* Similar to a pandas DataFrame, we can load a data set into an RDD, and then run any of the methods accesible to that object.

#### PySpark
While the Spark toolkit is in Scala, a language that compiles down to bytecode for the JVM, the open source community has developed a wonderful toolkit called [PySpark](https://spark.apache.org/docs/0.9.0/python-programming-guide.html) that allows us to interface with RDDs in Python. Thanks to a library called [Py4J](https://github.com/bartdag/py4j), Python can interface with Java objects (in our case RDDs). Py4J is also one of the tools that makes PySpark work.<br>

In this mission, we'll work with a data set containing the names of all of the guests who have appeared on [The Daily Show](https://en.wikipedia.org/wiki/The_Daily_Show).<br>

To start off, we'll load the data set into an RDD. We're using the `TSV` version of [FiveThirtyEight's data set](https://github.com/fivethirtyeight/data/tree/master/daily-show-guests). TSV files use a tab character (`"\t"`) as the delimiter, instead of the comma (`","`) that CSV files use.

### Install PySpark
* [Get Started with PySpark and Jupyter Notebook in 3 Minutes](https://blog.sicara.com/get-started-pyspark-jupyter-guide-tutorial-ae2fe84f594f)

In [2]:
import pyspark

In [3]:
sc = pyspark.SparkContext()

In [4]:
raw_data = sc.textFile("data/daily_show.tsv")
raw_data.take(5)

['YEAR\tGoogleKnowlege_Occupation\tShow\tGroup\tRaw_Guest_List',
 '1999\tactor\t1/11/99\tActing\tMichael J. Fox',
 '1999\tComedian\t1/12/99\tComedy\tSandra Bernhard',
 '1999\ttelevision actress\t1/13/99\tActing\tTracey Ullman',
 '1999\tfilm actress\t1/14/99\tActing\tGillian Anderson']

## SparkContext

In Spark, the `SparkContext` object manages the connection to the clusters, and coordinates the running of processes on those clusters. More specifically, it connects to the cluster managers. The cluster managers control the executors that run the computations. Here's a diagram from the Spark documentation that will help you visualize the architecture:

![](https://spark.apache.org/docs/1.1.0/img/cluster-overview.png)

We automatically have access to the SparkContext object `sc`. We then run the following code to read the TSV data set into an RDD object `raw_data`:

```python
raw_data = sc.textFile("daily_show.tsv")
```

The RDD object `raw_data` closely resembles a list of string objects, with one object for each line in the data set. We then use the `take()` method to print the first five elements of the RDD:

```python
raw_data.take(5)
```

To explore the other methods an RDD object has access to, check out the [PySpark documentation](https://spark.apache.org/docs/1.1.1/api/python/pyspark.rdd.RDD-class.html#take). `take(n)` will return the first `n` elements of the RDD.

## Lazy Evaluation

You may be wondering why, if an RDD resembles a Python list, we don't just use bracket notation to access elements in the RDD.<br>

The answer is that Spark distributes RDD objects across many partitions, and **the RDD object is specifically designed to handle distributed data. We can't rely on the standard implementation of a list for these reasons**.<br>

Spark offers many advantages over regular Python, though. For example, **thanks to RDD [abstraction](https://en.wikipedia.org/wiki/Abstraction_%28software_engineering%29), you can run Spark locally on your own computer**. 
* Spark will simulate distributing your calculations over many machines **by automatically slicing your computer's memory into partitions**.

Spark's RDD implementation also lets us **evaluate code "lazily,"** meaning **we can postpone running a calculation until absolutely necessary**. On the previous screen, Spark waited to load the TSV file into an RDD until `raw_data.take(5)` executed. ** When our code called `raw_data = sc.textFile("dail_show.tsv")`, Spark created a pointer to the file, but didn't actually read it into `raw_data` until `raw_data.take(5)` needed that variable to run its logic.

The advantage of "lazy" evaluation is that 
### we can build up a queue of tasks and let Spark optimize the overall workflow in the background. 

In regular Python, the interpreter can't do much workflow optimization. We'll see more examples of lazy evaluation later on.

## Pipelines

### While Spark borrowed heavily from Hadoop's MapReduce pattern, it's still quite different in many ways. 
If you have experience with Hadoop and traditional MapReduce, you may want to read this great [post by Cloudera](http://blog.cloudera.com/blog/2014/09/how-to-translate-from-mapreduce-to-apache-spark/) about the difference between them. Don't worry if you've never worked with MapReduce or Hadoop before; we'll cover the concepts you need to know in this course.<br>

### The key idea to understand when working with Spark is data pipelining.

Every operation or calculation in Spark is essentially a series of steps that we can chain together and run in succession to form a **pipeline**. Each step in the **pipeline** returns either 
* a Python value (such as an integer), 
* a Python data structure (such as a dictionary), 
* or an RDD object. 

We'll start with the `map()` function.

### `Map()`

The `map(f)` function applies the function `f` to every element in the RDD. Because RDDs are iterable objects (like most Python objects), Spark runs function `f` on each iteration and returns a new RDD.<br>

We'll walk through an example of a `map` function so you can get a better sense of how it works. If you look carefully, you'll see that `raw_data` is in a format that's hard to work with. While the elements are currently all `strings`, we'd like to convert each of them into a `list` to make the data more manageable. To do this the traditional way, we would:

```python
1. Use a 'for' loop to iterate over the collection
2. Split each `string` on the delimiter
3. Store the result in a `list`
```

### Let's see how we can use map to do this with Spark instead.

In the code cell:

```python
1. Call the RDD function `map()` to specify we want to apply the logic in the parentheses to every line in our data set.
2. Write a lambda function that splits each line using the tab delimiter (\t), and assign the resulting RDD to `daily_show`.
3. Call the RDD function `take()` on `daily_show` to display the first five elements (or rows) of the resulting RDD.
```

We call the `map(f)` function a transformation step. It requires either a named or lambda function `f`.

In [5]:
daily_show = raw_data.map(lambda line: line.split('\t'))
daily_show.take(5)
# Hit check to see the output

[['YEAR', 'GoogleKnowlege_Occupation', 'Show', 'Group', 'Raw_Guest_List'],
 ['1999', 'actor', '1/11/99', 'Acting', 'Michael J. Fox'],
 ['1999', 'Comedian', '1/12/99', 'Comedy', 'Sandra Bernhard'],
 ['1999', 'television actress', '1/13/99', 'Acting', 'Tracey Ullman'],
 ['1999', 'film actress', '1/14/99', 'Acting', 'Gillian Anderson']]

## Python and Scala, Friends Forever

One of the wonderful features of PySpark is the ability to separate our logic - which we prefer to write in Python - from the actual data transformation. In the previous code cell, we wrote this lambda function in Python code:

```python
raw_data.map(lambda line: line.split('\t'))
```

Even though the function was in Python, we also took advantage of Scala when Spark actually ran the code over our RDD. This is the power of PySpark. 
### Without learning any Scala, we get to harness the data processing performance gains from Spark's Scala architecture. 
Even better, when we ran the following code, it returned the results to us in Python-friendly notation:

```python
daily_show.take(5)
```

#### Transformations and Actions

There are two types of methods in Spark:

```python
1. Transformations - map(), reduceByKey()
2. Actions - take(), reduce(), saveAsTextFile(), collect()
```

Transformations are lazy operations that always return a reference to an RDD object. Spark doesn't actually run the transformations, though, until an action needs to use the RDD resulting from a transformation. 
* Any function that returns an RDD is a transformation, 
* and any function that returns a value is an action. 

These concepts will become more clear as we work through this lesson and practice writing PySpark code.

#### Immutability

You may be wondering why we couldn't just split each `string` in place, instead of creating a new object `daily_show`. In Python, we could have modified the collection element-by-element in place, without returning and assigning the results to a new object.<br>

RDD objects are [immutable](https://www.quora.com/Why-is-a-spark-RDD-immutable), meaning that **we can't change their values once we've created them**. 
* In Python, list and dictionary objects are mutable (we can change their values), while tuple objects are immutable. 
* The only way to modify a tuple object in Python is to create a new tuple object with the necessary updates. 

### Spark uses the immutability of RDDs to enhance calculation speeds. 
The mechanics of how it does this are outside the scope of this lesson.

## `ReduceByKey()`

We'd like to tally up the number of guests who have appeared on The Daily Show during each year. If `daily_show` were a list of lists, we could write the following Python code to achieve this result:

```python
tally = dict()
for line in daily_show:
  year = line[0]
  if year in tally.keys():
    tally[year] = tally[year] + 1
  else:
    tally[year] = 1
```
            
The keys in `tally` will be the years, and the values will be the totals for the number of lines associated with each year.<br>

To achieve the same result with Spark, we'll have to use a `Map` step, then a `ReduceByKey` step.

In [19]:
daily_show.take(5)

[['YEAR', 'GoogleKnowlege_Occupation', 'Show', 'Group', 'Raw_Guest_List'],
 ['1999', 'actor', '1/11/99', 'Acting', 'Michael J. Fox'],
 ['1999', 'Comedian', '1/12/99', 'Comedy', 'Sandra Bernhard'],
 ['1999', 'television actress', '1/13/99', 'Acting', 'Tracey Ullman'],
 ['1999', 'film actress', '1/14/99', 'Acting', 'Gillian Anderson']]

In [22]:
tally_ = daily_show.map(lambda x: (x[0], 1))
print(tally_)
tally_.take(5)

PythonRDD[17] at RDD at PythonRDD.scala:48


[('YEAR', 1), ('1999', 1), ('1999', 1), ('1999', 1), ('1999', 1)]

In [23]:
tally = daily_show.map(lambda x: (x[0], 1)).reduceByKey(lambda x,y: x+y)
print(tally)
tally.take(5)

PythonRDD[23] at RDD at PythonRDD.scala:48


[('YEAR', 1), ('2002', 159), ('2003', 166), ('2004', 164), ('2007', 141)]

## Explanation

You may have noticed that printing `tally` didn't return the histogram we were hoping for. Because of lazy evaluation, PySpark delayed executing the `map` and `reduceByKey` steps until we actually need them. Before we use `take()` to preview the first few elements in tally, we'll walk through the code we just wrote.

```python
daily_show.map(lambda x: (x[0], 1)).reduceByKey(lambda x, y: x+y)
```

During the map step, we used a lambda function to create a tuple consisting of:
```
key: x[0] (the first value in the list)
value: 1 (the integer)
```
Our high-level strategy was **to create a tuple with the key representing the year, and the value representing 1**. After running the map step, Spark will **maintain in memory a list of tuples resembling the following**:

```python
('YEAR', 1)
('1991', 1)
('1991', 1)
('1991', 1)
('1991', 1)
...
```

We'd like to reduce that down to:

```python
('YEAR', 1)
('1991', 4)
...
```

`reduceByKey(f)` combines tuples with the same key using the function we specify, `f`.<br>

To see the results of these two steps, we'll use the `take` command, **which forces lazy code to run immediately**. Because `tally` is an RDD, we can't use Python's len function to find out how many elements are in the collection. Instead, we'll need to use the RDD `count()` function.

In [24]:
print(tally.count)

<bound method RDD.count of PythonRDD[23] at RDD at PythonRDD.scala:48>


In [ ]:
tally.take(tally.count())

## Filter

Unlike pandas, Spark knows nothing about column headers, and didn't set them aside. We need a way to remove the element `('YEAR', 1)` from our collection. We'll need a workaround, though, because RDD objects are immutable once we create them. The only way to remove that tuple is to create a new RDD object that doesn't have it.<br>

Spark comes with a `filter(f)` function that creates a new RDD by filtering an existing one for specific criteria. If we specify a function `f` that returns a binary value, `True` or `False`, the resulting RDD will consist of elements where the function evaluated to `True`. You can read more about the `filter` function in the [Spark documentation](https://spark.apache.org/docs/1.1.1/api/python/pyspark.rdd.RDD-class.html#filter).

* Write a function named `filter_year` that we can use to filter out the element that begins with the text `YEAR`, instead of an actual year.

In [28]:
def filter_year(line):
    # Write your logic here
    if line[0] == 'YEAR':
        return False
    return True

filtered_daily_show = daily_show.filter(lambda line: filter_year(line))

## Practice with Pipelines

To flex Spark's muscles, we'll demonstrate how to chain together a series of data transformations into a pipeline, and observe Spark managing everything in the background. The developers who wrote Spark had this functionality in mind, and optimized it for running tasks in succession.<br>

**Before Spark came along, running lots of tasks in succession in Hadoop was incredibly time consuming**. Hadoop had to write intermediate results to disk, and wasn't aware of the full pipeline. Thanks to its aggressive approach to memory use and well-architected core, Spark improves on Hadoop's turnaround time significantly. If you're curious, you can read more about this topic in a [Quora thread](http://qr.ae/RHWrT2).<br>

```
TL;DR - Conceptually DAG model is a strict generalization of MapReduce model. DAG-based systems like Spark and Tez that are aware of the whole DAG of operations can do better global optimizations than systems like Hadoop MapReduce which are unaware of the DAG to be executed.
```

In the following code cell, we'll filter out actors for whom the profession is blank, lowercase each profession, generate a histogram of professions, and output the first five tuples in the histogram.

In [29]:
filtered_daily_show.filter(lambda line: line[1] != '') \
                   .map(lambda line: (line[1].lower(), 1)) \
                   .reduceByKey(lambda x,y: x+y) \
                   .take(5)

[('actor', 596),
 ('film actress', 21),
 ('model', 9),
 ('stand-up comedian', 44),
 ('actress', 271)]

## Next Steps

In this mission, we introduced the MapReduce paradigm, the fundamentals of Spark, and PySpark data transformations. Next, you'll install Spark and PySpark on your own machine.

In [ ]:
sc.stop()